In [10]:
#!pip install azureml-contrib-fairness
#!pip install fairlearn==0.4.6
#!pip install raiwidgets
#pip install azureml-dataprep[pandas]

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Environment
from azureml.core import *
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.core import Workspace, Dataset, Datastore
import os, shutil
from azureml.core import Dataset
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.core import Workspace, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace,RunConfiguration
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep
from azureml.train.automl.utilities import get_primary_metrics
from azureml.data.datapath import DataPath
from azureml.core.datastore import Datastore
from azureml.pipeline.core import InputPortBinding
from azureml.core import Run
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.pipeline.core import PipelineParameter
from azureml.core import Dataset
from azureml.core import Environment
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.data import OutputFileDatasetConfig
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive import *
from azureml.widgets import RunDetails
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule
from azureml.core import Run
from azureml.core import get_run
import pandas as pd
import numpy as np
from fairlearn.reductions import GridSearch
from fairlearn.reductions import DemographicParity
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from interpret.ext.blackbox import MimicExplainer
from interpret.ext.glassbox import LGBMExplainableModel
from raiwidgets import FairnessDashboard
from raiwidgets import ExplanationDashboard
from azureml.pipeline.core import Pipeline
from azureml.core import Environment


In [11]:
# Environnement
subscription_id = 'a0f4cddc-a66a-4dcc-9df7-ccbd7f81bf7b'
resource_group = '-----------'
workspace_name = '---------'
compute_engine = 'jucalcul1'
cluster_name = 'jucalcul1'
dataset_name = 'trainingdataset'
experiment_name = "25082021"

# if not exist yet
#ws = Workspace.create(name='myworkspace',
#                      subscription_id='<azure-subscription-id>',
#                      resource_group='myresourcegroup',
#                      create_resource_group=True,
#                      location='eastus2')

ws = Workspace.get(subscription_id = "--------------",
                   resource_group = "------------------",
                   name = "-----------")

In [12]:
# Compute 
components_dir = '.'
os.makedirs(components_dir, exist_ok=True)


# Verify that the compute cluster exists
try:
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster : ', cluster_name)
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=3)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex, cluster_name)


Found existing cluster :  jucalcul1


In [15]:
# environment preparation : 
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment 

aml_run_config = RunConfiguration()
aml_run_config.target = compute_engine

USE_CURATED_ENV = True
if USE_CURATED_ENV :
    curated_environment = Environment.get(workspace=ws, name="AzureML")
    aml_run_config.environment = curated_environment
else:
    aml_run_config.environment.python.user_managed_dependencies = False
    
    # Add some packages relied on by data prep step
    aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
        conda_packages=['pandas','scikit-learn'], 
        pip_packages=['azureml-sdk', 'azureml-dataprep[fuse,pandas]','pandas'], 
        pin_sdk_version=False)

In [17]:
# Step 0 environnement
data_folder = os.path.join(os.getcwd(), 'data/')
os.makedirs(data_folder, exist_ok=True)
sklearn_env = Environment.from_conda_specification(name = 'sklearn_envv2', file_path = './conda_dependencies.yml')

# Step 0 data ingestion
raw_ds = Dataset.get_by_name(ws, name=dataset_name, version='latest') 
datastore = ws.get_default_datastore()

# allow to use output1 in input2
fileConfig = OutputFileDatasetConfig(name='file_dataset')

# Step 1 data preparation
step1 = PythonScriptStep(name = 'prepare data',
                         source_directory = 'scripts',
                         script_name = 'data_prep.py',
                         compute_target = compute_engine,
                         runconfig=aml_run_config,
                         allow_reuse=True,
                         arguments = ['--raw-ds', raw_ds.as_named_input('raw_data'), # has_named_input is the method for tabular dataset 
                                      '--output-dir', fileConfig])

# Step 2 : training 
train_src = ScriptRunConfig(source_directory = 'scripts',
                            script = 'model_script2.py',
                            compute_target = compute_engine,
                            arguments = ["--input-dir", fileConfig.as_input()],
                            environment = sklearn_env)


# Step 2 : hyperparameter 
param_sampling = RandomParameterSampling( {
    "--n_estimators": choice(15, 50, 100, 200, 300),
    "--criterion": choice("gini", "entropy"),
    "--max_depth": choice(2, 8, 16)
    }
)

hd_config = HyperDriveConfig(run_config=train_src, 
                             hyperparameter_sampling=param_sampling,
                             primary_metric_name='precision', 
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs=1, # 1 for testing 
                             max_concurrent_runs=4)


saved_model = PipelineData(name='saved_model',
                           datastore=datastore,
                           pipeline_output_name='model_output',
                           training_output=TrainingOutput(type = "Model", metric="Precision", model_file="outputs/model/save_model.pkl"))

metrics_data = PipelineData(name='metrics_data', 
                            datastore=datastore,
                            pipeline_output_name='metrics_output',
                            training_output=TrainingOutput(type='Metrics'))

                                
 
hd_step = HyperDriveStep(
    allow_reuse=True,
    name='hyperparameters',
    outputs = [metrics_data, saved_model],
    hyperdrive_config=hd_config,
                             )



# step 3 : register best model
# best model is retrieve directly from PipelineData saved model
register_model = PythonScriptStep(name = 'Model Registration',
                                  script_name = 'scripts/register_model.py',
                                  inputs = [metrics_data, saved_model],
                                  compute_target = compute_engine,
                                  allow_reuse=True,
                                  arguments = ["--saved-model", saved_model,
                                                "--model-file", "model.pkl",
                                                "--model-description","pipeline with hyperameters",
                                                "--model-name","best_model_2508"]
                                  )


# step 3 : publish pipeline
pipeline_training = [step1, hd_step, register_model]
pipeline = Pipeline(workspace=ws, steps=[pipeline_training])
pipeline_run = pipeline.submit(experiment_name, regenerate_outputs=True)

#published_pipeline1 = pipeline.publish(
#                        name="Template_Pipeline_Notebook",
#                        description="Published Pipeline Description",
#                        version="1.0")


# step 3 : schedule pipeline run every day
#recurrence = ScheduleRecurrence(frequency='Day', interval=1)
#recurring_schedule = Schedule.create(ws, name='DailySchedule', 
#                            description='Once a day',
#                            pipeline_id=published_pipeline1.id, 
#                            experiment_name= experiment_name, 
#                            recurrence=recurrence)


No Python version provided, defaulting to "3.6.2"
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


Created step prepare data [045bfae9][c4539b2b-4fe0-4c00-a14d-7d4f556a6088], (This step will run and generate new outputs)Created step hyperparameters [8bc0df80][b71edf20-6a18-49c9-a42b-3ff77424725d], (This step will run and generate new outputs)

Created step Model Registration [7eb7e2cd][bdf0d52e-da78-4f9d-9a03-d6770386bb06], (This step will run and generate new outputs)
Submitted PipelineRun 4a31bd0a-4c28-4324-bef6-a1b12296f79d
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/4a31bd0a-4c28-4324-bef6-a1b12296f79d?wsid=/subscriptions/a0f4cddc-a66a-4dcc-9df7-ccbd7f81bf7b/resourcegroups/ressourcesecu/workspaces/deuxiemeespace&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
